In [2]:
import pathlib
import pandas as pd
import numpy as np

from matplotlib import pyplot as plt

In [35]:
AMD_CSV_PATH = pathlib.Path('../../docs/2017_09_pact/data/case-study-a/cgo17-amd.csv')
NVIDIA_CSV_PATH = pathlib.Path('../../docs/2017_09_pact/data/case-study-a/cgo17-nvidia.csv')

assert AMD_CSV_PATH.is_file()
assert NVIDIA_CSV_PATH.is_file()

amd_df = pd.read_csv(AMD_CSV_PATH)
nvidia_df = pd.read_csv(NVIDIA_CSV_PATH)
amd_df.drop('Unnamed: 0', axis='columns', inplace=True)
nvidia_df.drop('Unnamed: 0', axis='columns', inplace=True)

In [40]:
df = amd_df.join(nvidia_df, lsuffix='_amd', rsuffix='_nvidia')
assert all(x == y for x, y in df[['benchmark_amd', 'benchmark_nvidia']].values)
assert all(x == y for x, y in df[['dataset_amd', 'dataset_nvidia']].values)

df['program:opencl_kernel_name'] = [b.split('-')[-1] for b in df['benchmark_amd']]
df['program:benchmark_name'] = [b.split('-')[-2] for b in df['benchmark_amd']]
df['program:benchmark_suite_name'] = ['-'.join(b.split('-')[:-2]) for b in df['benchmark_amd']]

df.rename({
    'dataset_amd': 'data:dataset_name',
}, axis='columns', inplace=True)
df.drop([
    'benchmark_amd', 
    'benchmark_nvidia',
    'dataset_nvidia',
], axis='columns', inplace=True)
df

,data:dataset_name,comp_amd,rational_amd,mem_amd,localmem_amd,coalesced_amd,atomic_amd,transfer_amd,wgsize_amd,oracle_amd,...,transfer_nvidia,wgsize_nvidia,oracle_nvidia,runtime_cpu_nvidia,runtime_gpu_nvidia,src_nvidia,seq_nvidia,program:opencl_kernel_name,program:benchmark_name,program:benchmark_suite_name
0,default,98,8,13,11,2,0,2048,255,GPU,...,2048,255,GPU,3.291073,0.152038,"__kernel void A(int a, const __global float4* ...","[129 129 129 ..., 0 127 0]",binomial_options,BinomialOption,amd-app-sdk-3.0
1,default,12,2,6,0,6,0,131072,256,CPU,...,131072,256,GPU,0.121940,0.057834,"__kernel void A(__global uint* a, const uint b...","[129 129 129 ..., 0 127 0]",bitonicSort,BitonicSort,amd-app-sdk-3.0
2,default,220,4,3,0,0,0,3145728,256,GPU,...,3145728,256,GPU,6.076052,1.997728,"__kernel void A(const __global float4* a, int ...","[129 129 129 ..., 26 0 127]",blackScholes,BlackScholes,amd-app-sdk-3.0
3,default,8,0,4,0,4,0,4096,256,CPU,...,4096,256,GPU,0.130003,0.062024,"__kernel void A(__global float* a, __const int...","[129 129 129 ..., 0 127 0]",fastWalshTransform,FastWalshTransform,amd-app-sdk-3.0
4,default,11,1,5,0,0,0,524288,256,CPU,...,524288,256,CPU,0.283447,0.604686,"__kernel void A(__global uint* a, __global uin...","[129 129 129 ..., 127 0 127]",floydWarshallPass,FloydWarshall,amd-app-sdk-3.0
5,default,163,1,12,0,0,0,49152,64,CPU,...,49152,64,GPU,0.277644,0.124211,"__kernel void A(__global float4* a, __global f...","[ 1 9 1 ..., 0 127 0]",mmmKernel,MatrixMultiplication,amd-app-sdk-3.0
6,default,42,0,16,8,2,0,32768,256,CPU,...,32768,256,GPU,0.156639,0.087846,"__kernel void A(__global float4* a, __global f...","[129 129 129 ..., 0 127 0]",matrixTranspose,MatrixTranspose,amd-app-sdk-3.0
7,default,36,10,12,8,2,0,4096,640,CPU,...,4096,1024,GPU,0.484234,0.049376,"__kernel void A(__global float* a, __global fl...","[129 129 129 ..., 0 127 0]",group_prefixSum,PrefixSum,amd-app-sdk-3.0
8,default,6,3,7,4,2,0,8208,256,CPU,...,8208,256,GPU,0.161025,0.070912,"__kernel void A(__global uint4* a, __global ui...","[129 129 129 ..., 0 127 0]",reduce,Reduction,amd-app-sdk-3.0
9,default,32,3,21,14,4,0,263168,128,CPU,...,263168,128,GPU,0.336185,0.038630,"__kernel void A(__global float* a, __global fl...","[129 129 129 ..., 127 0 127]",ScanLargeArrays,ScanLargeArrays,amd-app-sdk-3.0


In [9]:
src_df = runtimes_df[runtimes_df.cf == 1][['kernel', 'src']]
src_df.rename({
    'kernel': 'program:opencl_kernel_name',
    'src': 'program:opencl_kernel_src',
}, axis='columns', inplace=True)
src_df.set_index('program:opencl_kernel_name', inplace=True)
src_df.sort_index(inplace=True)
# src_df

NameError: name 'runtimes_df' is not defined